# Finetune `meta-llama/Meta-Llama-3-8B-Instruct` on an EC2 instance using `Unsloth`
---

Unsloth makes finetuning large language models like Llama-3, Mistral, Phi-4 and Gemma 2x faster, use 70% less memory, and with no degradation in accuracy!

**Note**: ***This notebook is run on a `g6e.12xlarge` instance. Follow the prerequisite steps [here](README.md)***

In this example, we will be fine tuning the llama3 8b instruct model. There are several 4bit pre quantized models that `unsloth` provides that are not gated. This supports 4x faster downloading with no OOMs. In this case, we will be using the standard `meta-llama/Meta-Llama-3-8B-Instruct` model from hugging face. 

In [1]:
import os
import logging
import globals as g
from dotenv import load_dotenv
from unsloth import to_sharegpt
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import standardize_sharegpt
from ec2_metrics import EC2MetricsCallback

# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove existing handlers
logger.handlers.clear()

# Add a simple handler
handler = logging.StreamHandler()
formatter = logging.Formatter('[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


2025-03-17 02:08:26,476	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Load environment variables from .env file
import getpass
load_dotenv()
if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your HuggingFace token: ")
hf_token = os.getenv("HF_TOKEN")

if not os.getenv("HF_MODEL_ID2"):
    hf_model_id  = input("Enter the model id to use for fine-tuning (e.g. meta-llama/Llama-3.1-8B-Instruct): ")
else:
    hf_model_id = os.getenv("HF_MODEL_ID2")
logger.info(f"hf_model_id={hf_model_id}")


[2025-03-17 02:08:31,854] p63306 {730293.py:12} INFO - hf_model_id=meta-llama/Llama-3.2-1B-Instruct


In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

DATASET_OF_INTEREST: str = 'mteb/banking77'

ALPACA_PROMPT: str = """Below are the questions asked by customs, categorized according to the provided examples.
### Question: 
{}
### Category_id: 
{}
### Category: 
{}"""

In [4]:
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = hf_model_id,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = hf_token # use one if using gated models like meta-llama/Llama-2-7b-hf
    )
except Exception as e:
    logger.error(f"Error occurred while loading the model: {e}")
    raise

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.045 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


## Data Prep

We now use the Alpaca dataset from vicgalle, which is a version of 52K of the original Alpaca dataset generated from GPT4. You can replace this code section with your own data prep.

In [18]:
dataset = load_dataset(DATASET_OF_INTEREST, split="train")
logger.info(f"Columns in the dataset: {dataset.column_names}")

[2025-03-17 02:12:48,069] p63306 {2022229014.py:2} INFO - Columns in the dataset: ['text', 'label', 'label_text']


In [19]:
dataset = to_sharegpt(
    dataset,
    merged_prompt="Question: {text}[[\nCategory ID is:\n{label}]]",
    output_column_name="label_text",
    # conversation_extension=3,
)

In [20]:
# Use the standardize_sharegpt function to just make the dataset in a correct format for finetuning
dataset = standardize_sharegpt(dataset)

In [21]:
from pprint import pprint
pprint(dataset[:3])

{'conversations': [[{'content': 'Below is a user query:\n'
                                "('I am still waiting on my card?',)\n"
                                '\n'
                                'What is the correct intent?',
                     'role': 'user'},
                    {'content': 'card_arrival', 'role': 'assistant'}],
                   [{'content': 'Below is a user query:\n'
                                '("What can I do if my card still hasn\'t '
                                'arrived after 2 weeks?",)\n'
                                '\n'
                                'What is the correct intent?',
                     'role': 'user'},
                    {'content': 'card_arrival', 'role': 'assistant'}],
                   [{'content': 'Below is a user query:\n'
                                "('I have been waiting over a week. Is the "
                                "card still coming?',)\n"
                                '\n'
                       

In [23]:
chat_template = """Below are the questions asked by customs, categorized according to the provided examples.

### Question:  
{INPUT}  
### Category:  
{OUTPUT}  

### Question:  
{INPUT}  
### Category:  
{OUTPUT}  
"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|██████████| 10003/10003 [00:00<00:00, 30515.69 examples/s]


In [24]:
%%time
# train the model
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 600,
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    callbacks=[EC2MetricsCallback],
)

Converting train dataset to ChatML (num_proc=2): 100%|██████████| 10003/10003 [00:00<00:00, 24652.57 examples/s]
Applying chat template to train dataset (num_proc=2): 100%|██████████| 10003/10003 [00:01<00:00, 7264.93 examples/s]
Tokenizing train dataset (num_proc=2): 100%|██████████| 10003/10003 [00:00<00:00, 14084.65 examples/s]

CPU times: user 816 ms, sys: 337 ms, total: 1.15 s
Wall time: 5.14 s


In [25]:
%%time
# this will initiate the training process and also log the EC2 utilization metrics, such as the GPU
# utilization, CPU utilization, etc.
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,003 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 600
 "-____-"     Number of trainable parameters = 11,272,192
[2025-03-17 02:13:29,984] p63306 {ec2_metrics.py:184} INFO - Training started. Initiating EC2 metrics collection.
[2025-03-17 02:13:29,985] p63306 {ec2_metrics.py:170} INFO - Writing header: ['timestamp', 'cpu_percent_mean', 'memory_percent_mean', 'memory_used_mean', 'gpu_utilization_mean', 'gpu_memory_used_mean', 'gpu_memory_free_mean', 'gpu_memory_total_mean']
[2025-03-17 02:13:29,986] p63306 {ec2_metrics.py:41} INFO - Starting collection
[2025-03-17 02:13:30,191] p63306 {ec2_metrics.py:143} INFO - Starting daemon collector to run in background


Step,Training Loss
1,4.303300
2,4.447600
3,4.394200
4,4.303900
5,4.112000
6,3.756500
7,3.374200
8,2.861200
9,2.381700
10,2.121500


[2025-03-17 02:18:29,730] p63306 {ec2_metrics.py:191} INFO - Training ended. Stopping EC2 metrics collection.
[2025-03-17 02:18:29,731] p63306 {ec2_metrics.py:33} INFO - Stopped collection


CPU times: user 4min 58s, sys: 5.76 s, total: 5min 4s
Wall time: 5min


### Log the trainer stats
---

In this step, we log some of the trainer stats, such as the number of global steps it took to get to a specific training loss, the train runtime, samples per second, steps per second, etc.

In [32]:
# Format the training stats in a readable way
output_text = f"""Training Statistics:
Global Steps: {trainer_stats.global_step}
Training Loss: {trainer_stats.training_loss:.4f}

Metrics:
- Train Runtime: {trainer_stats.metrics['train_runtime']:.3f} seconds
- Training Samples/Second: {trainer_stats.metrics['train_samples_per_second']:.3f}
- Training Steps/Second: {trainer_stats.metrics['train_steps_per_second']:.3f}
- Total FLOPS: {trainer_stats.metrics['total_flos']:.2e}
- Final Train Loss: {trainer_stats.metrics['train_loss']:.4f}
"""

# Save to a text file
with open(os.path.join(g.RESULTS_DIR, g.TRAINING_STATS), 'w') as f:
    f.write(output_text)

In [33]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "I am still waiting on my card?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

card_arrival  
<|eot_id|>


In [34]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "I am still waiting on my card?"},
    {"role": "assistant", "content": "card_arrival"},
    {"role": "user",      "content": "I see a charge on my credit card statement but I paid on time, why?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer) #, skip_prompt = True)
# _ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

# Generate response
output_ids = model.generate(input_ids, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)

# Decode the output into a string
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print or store the result
print("Generated Output:", output_text)

Generated Output: Below are the questions asked by customs, categorized according to the provided examples.

### Question:  
I am still waiting on my card?  
### Category:  
card_arrival  

### Question:  
I see a charge on my credit card statement but I paid on time, why?  
### Category:  
card_payment_not_recognised  



In [39]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "I am still waiting on my card?"},
    {"role": "assistant", "content": "card_arrival"},
    {"role": "user",      "content": "Do you have a branch in Timbuktu?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer) #, skip_prompt = True)
# _ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

# Generate response
output_ids = model.generate(input_ids, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)

# Decode the output into a string
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print or store the result
print("Generated Output:", output_text)

Generated Output: Below are the questions asked by customs, categorized according to the provided examples.

### Question:  
I am still waiting on my card?  
### Category:  
card_arrival  

### Question:  
Do you have a branch in Timbuktu?  
### Category:  
branches  



In [40]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "I am still waiting on my card?"},
    {"role": "assistant", "content": "card_arrival"},
    {"role": "user",      "content": "I lost my card and my replacement card has not arrived."},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer) #, skip_prompt = True)
# _ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

# Generate response
output_ids = model.generate(input_ids, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)

# Decode the output into a string
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print or store the result
print("Generated Output:", output_text)

Generated Output: Below are the questions asked by customs, categorized according to the provided examples.

### Question:  
I am still waiting on my card?  
### Category:  
card_arrival  

### Question:  
I lost my card and my replacement card has not arrived.  
### Category:  
card_arrival  



In [16]:
# save the model
model.save_pretrained("lora_model2")  # Local saving
tokenizer.save_pretrained("lora_model2")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

## Ollama Support [Optional]

Unsloth now allows you to automatically finetune and create a Modelfile, and export to Ollama! This makes finetuning much easier and provides a seamless workflow from Unsloth to Ollama!

Let's first install Ollama!

In [17]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%####################                                                36.6%#############################################               83.3%
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> Enabling and starting ollama service...
>>> NVIDIA GPU installed.


In [21]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.42 out of 15.01 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  0%|          | 0/32 [00:00<?, ?it/s]

make: Entering directory '/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/llama.cpp'
make: Leaving directory '/home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/llama.cpp'
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend



Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.


-- x86 detected
-- Adding CPU backend variant ggml-cpu: -march=native 
-- Configuring done (0.2s)
-- Generating done (0.3s)
-- Build files have been written to: /home/ubuntu/yf265/spring-2025-lab07-Yuting-Fan-265/llama.cpp/build
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.16 out of 15.01 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  0%|          | 0/32 [00:00<?, ?it/s]


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 576 vs 470

In [ ]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3) 

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


KeyboardInterrupt: 

Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [ ]:
!ollama create unsloth_model -f ./model/Modelfile

In [ ]:
# run inference against the model
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
    }'

In [ ]:
# run inference against the model
import json
result = subprocess.run(
    [
        "curl",
        "http://localhost:11434/api/generate",
        "-d",
        '{"model": "unsloth_model", "prompt": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8,", "stream": false}',
    ],
    capture_output=True,
    text=True,
)

response_data = json.loads(result.stdout)
print(f"Response generated: {response_data['response']}")

